Read data file and create data schema

In [1]:
import pyspark
from pyspark.sql.types import *

#create session
appName = "data preprocessing in Spark"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

flightSchema = StructType([
  StructField("DayofMonth", IntegerType(), False),
  StructField("DayOfWeek", IntegerType(), False),
  StructField("Carrier", StringType(), False),
  StructField("OriginAirportID", IntegerType(), False),
  StructField("DestAirportID", IntegerType(), False),
  StructField("DepDelay", IntegerType(), False),
  StructField("ArrDelay", IntegerType(), False),
])

flights = spark.read.csv('dataset/raw-flight-data.csv', 
                         schema=flightSchema, header=True)
flights.show(2)

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
+----------+---------+-------+---------------+-------------+--------+--------+
only showing top 2 rows



In [2]:
airportSchema = StructType([
  StructField("airport_id", IntegerType(), False),
  StructField("city", StringType(), False),
  StructField("state", StringType(), False),
  StructField("name", StringType(), False),
])

airports = spark.read.csv('dataset/airports.csv', header=True, 
                          schema=airportSchema)
airports.show(2)

+----------+-----------+-----+--------------------+
|airport_id|       city|state|                name|
+----------+-----------+-----+--------------------+
|     10165|Adak Island|   AK|                Adak|
|     10299|  Anchorage|   AK|Ted Stevens Ancho...|
+----------+-----------+-----+--------------------+
only showing top 2 rows



In [3]:
#Merge two dataFrame (flight and airports), and show how many flights from each City
flightsByOrigin = flights.join(airports,
                               flights.OriginAirportID == 
                               airports.airport_id).groupBy("city").count()
flightsByOrigin.sort("city").show(10)

+-----------+------+
|       city| count|
+-----------+------+
|Albuquerque| 17628|
|  Anchorage|  7777|
|    Atlanta|149970|
|     Austin| 27155|
|  Baltimore| 52005|
|     Boston| 67155|
|    Buffalo| 14863|
|    Burbank| 14418|
|  Charlotte| 76893|
|    Chicago|177845|
+-----------+------+
only showing top 10 rows



Handle duplicated data

In [4]:
#count the number of original data rows
n1 = flights.count()
print("number of original data rows: ", n1)
#count the number of data rows after deleting duplicated data
n2 = flights.dropDuplicates().count()
print("number of data rows after deleting duplicated data: ", n2)
n3 = n1 - n2
print("number of duplicated data: ", n3)

number of original data rows:  2719418
number of data rows after deleting duplicated data:  2696983
number of duplicated data:  22435


In [5]:
#After dropping duplicates, Merge two dataFrame (flight and airports), and show how many flights from each City
flightsdropDuplicates=flights.dropDuplicates()
airportsdropDuplicates=airports.dropDuplicates()
flightsByOrigin = flightsdropDuplicates.join(airports,
                               flights.OriginAirportID == 
                               airportsdropDuplicates.airport_id).groupBy("city").count()
flightsByOrigin.sort("city").show(10)

+-----------+------+
|       city| count|
+-----------+------+
|Albuquerque| 17536|
|  Anchorage|  7700|
|    Atlanta|148462|
|     Austin| 26960|
|  Baltimore| 51711|
|     Boston| 66567|
|    Buffalo| 14773|
|    Burbank| 14114|
|  Charlotte| 76383|
|    Chicago|176144|
+-----------+------+
only showing top 10 rows



In [6]:
df = spark.createDataFrame([("Rony",27, 168), 
                            ("Rony",15, 165), 
                            ("Rony",27, 168)], 
                           ["name","age","height"])
df.show()
df.dropDuplicates().show()

+----+---+------+
|name|age|height|
+----+---+------+
|Rony| 27|   168|
|Rony| 15|   165|
|Rony| 27|   168|
+----+---+------+

+----+---+------+
|name|age|height|
+----+---+------+
|Rony| 27|   168|
|Rony| 15|   165|
+----+---+------+



In [7]:
df.dropDuplicates(['name']).show()

+----+---+------+
|name|age|height|
+----+---+------+
|Rony| 27|   168|
+----+---+------+



Handle missing data

In [8]:
flightsNoMissingValue = flightsdropDuplicates.dropna(
    how="any", subset=["ArrDelay", "DepDelay"])# use how="all" for all column missing data
numberOfMissingValueAny = n1 - flightsNoMissingValue.count()
print("number of missing value rows: ", numberOfMissingValueAny)

number of missing value rows:  46233


In [9]:
#take mean value
flightsdropDuplicates.groupBy().avg("ArrDelay").show()
meanArrDelay = flightsdropDuplicates.groupBy().avg("ArrDelay").take(1)[0][0]
print("mean ArrDelay: ", meanArrDelay)
meanDepDelay = flightsdropDuplicates.groupBy().avg("DepDelay").take(1)[0][0]
print("mean DepDelay: ", meanDepDelay)
#drop duplicated data and fill missing data with mean value
flightsCleanData=flightsdropDuplicates.fillna(
    {'ArrDelay': meanArrDelay, 'DepDelay': meanDepDelay})

+------------------+
|     avg(ArrDelay)|
+------------------+
|6.7272897311633875|
+------------------+

mean ArrDelay:  6.7272897311633875
mean DepDelay:  10.618575625454712


Explore the statistic of our data

In [10]:
flightsCleanData.describe("ArrDelay","DepDelay").show()

+-------+-----------------+------------------+
|summary|         ArrDelay|          DepDelay|
+-------+-----------------+------------------+
|  count|          2696983|           2696983|
|   mean| 6.72087217457433|10.613481805409972|
| stddev|38.57879179454158|    36.04900147973|
|    min|              -94|               -63|
|    max|             1845|              1863|
+-------+-----------------+------------------+



In [11]:
correlation = flightsCleanData.corr('DepDelay', 'ArrDelay')
print("correlation between departure delay and arrival delay: ", 
      correlation)

correlation between departure delay and arrival delay:  0.9394056759019356
